## Preprocess things files

### Imports

In [1]:
import pandas as pd
import h5py as h5
import numpy as np
from pathlib import Path
from convergence.nsd import get_resource
import os
from tqdm import tqdm
import nibabel as nib

### Data

In [2]:
dataset_path = Path(os.getenv("NSD_DATASET")).parent
things_path = dataset_path / "things"
preprocessed_path = things_path / "preprocessed"
preprocessed_path.mkdir(exist_ok=True)

results_folder = Path(os.getenv("CONVERGENCE_RESULTS"))


filename_stimulus_metadata = "betas_csv/betas_csv/sub-0{subject_id}_StimulusMetadata.csv"
filename_voxel_metadata = "betas_csv/betas_csv/sub-0{subject_id}_VoxelMetadata.csv"
filename_response_data = "/mnt/tecla/Datasets/things/betas_csv/betas_csv/sub-0{subject_id}_ResponseData.h5"

filename_betas = "betas_vol/scalematched/sub-0{subject_id}/ses-things01/sub-0{subject_id}_ses-things01_run-01_betas.nii.gz"

### Functions

In [49]:
def build_voxel_roi_table(df_voxel: pd.DataFrame, hemisphere_threshold: int) -> pd.DataFrame:
    subject_id = df_voxel.subject_id.unique()
    assert len(subject_id) == 1
    subject_id = subject_id[0]

    glasser_rois = [roi for roi in list(df_voxel.columns) if roi.startswith("glasser")]
    glasser_dict = {k: i+1 for i, k in enumerate(glasser_rois)}

    id_vars = ['voxel_id', 'subject_id', 'voxel_x', 'voxel_y', 'voxel_z']

    df_glasser = df_voxel[id_vars + glasser_rois]
    # Pivot. Voxel x,y,z as index. columns moved to rows with a new column 'glasser' with the roi name
    df_glasser = df_glasser.melt(id_vars=id_vars, var_name='glasser', value_name='value')
    df_glasser = df_glasser.query('value > 0').sort_values('voxel_id').reset_index(drop=True).copy()
    df_glasser['roi'] = df_glasser['glasser'].map(glasser_dict)
    total_voxels = len(df_glasser)
    assert total_voxels == len(df_glasser.drop_duplicates(subset=['voxel_x', 'voxel_y', 'voxel_z', 'roi']))
    hcp = get_resource("hcp")
    df_glasser = df_glasser.merge(hcp[['roi', 'name']], on='roi')
    assert total_voxels == len(df_glasser)
    df_glasser = df_glasser.drop(columns=['glasser', 'value'])
    df_glasser.voxel_x.astype("uint8")
    df_glasser.voxel_y.astype("uint8")
    df_glasser.voxel_z.astype("uint8")
    df_glasser.roi.astype("uint16")
    df_glasser['name'] = df_glasser['name'].astype("string").astype("category")
    df_glasser.subject_id.astype("uint8").astype('category')
    
    df_glasser.voxel_id = df_glasser.voxel_id.astype("uint32")

    # Add hemisphere
    df_glasser['hemisphere'] = (df_glasser.voxel_x < hemisphere_threshold).replace({True: 'lh', False: 'rh'}).astype("string").astype("category")
    df_glasser.loc[df_glasser.hemisphere == 'lh', 'roi'] += 180

    assert df_glasser.voxel_id.is_unique
    df_glasser['voxel_roi_index'] = df_glasser.groupby('roi').cumcount().astype("uint32")
    assert len(df_glasser.drop_duplicates(['roi', 'voxel_roi_index'])) == total_voxels

    return df_glasser

def get_voxel_size(df_voxel: pd.DataFrame) -> tuple[int, int, int]:
    max_x = df_voxel.voxel_x.max() + 1
    max_y = df_voxel.voxel_y.max() + 1
    max_z = df_voxel.voxel_z.max() + 1
    return max_x, max_y, max_z

def get_dense_mask(df_roi_coordinates: pd.DataFrame, size: tuple[int, int, int]):
    dense_mask = np.zeros(size, dtype=np.uint16)
    for _, row in df_roi_coordinates.iterrows():
        dense_mask[row.voxel_x, row.voxel_y, row.voxel_z] = row.roi
    return dense_mask

def compute_voxel_tables(df_stimulus, df_roi_coordinates, file_betas) -> dict[str, np.ndarray]:
    n_trials = len(df_stimulus)
    response_data = file_betas['ResponseData']
    #block0_items = response_data['block0_items'][()][0] # 29431
    voxel_ids = response_data['block1_values'][()][:, 0]
    betas = response_data['block0_values'][()] # (211339, 9840) (n_voxels, n_trials)
    assert betas.shape == (len(voxel_ids), n_trials)

    voxel_count = df_roi_coordinates.groupby("roi").voxel_roi_index.count().to_dict()


    beta_tables = {}
    for roi, count in voxel_count.items():
        beta_tables[roi] = np.zeros((n_trials, count), dtype=np.float32)
    rois_dict = df_roi_coordinates[['voxel_id', 'roi', 'voxel_roi_index']].set_index("voxel_id").to_dict('index')

    assert len(voxel_ids) == betas.shape[0]
    for voxel_id, roi_betas in zip(voxel_ids, betas):
        if voxel_id not in rois_dict:
            continue
        roi = rois_dict[voxel_id]['roi']

        voxel_roi_index = rois_dict[voxel_id]['voxel_roi_index']
        beta_tables[roi][:, voxel_roi_index] = roi_betas



    for beta_table in beta_tables.values():
        assert (np.abs(beta_table.sum(axis=0)) == 0).sum() == 0, "Possible empty voxel"
    return beta_tables



### Compute betas by roi

In [ ]:
for subject_id in tqdm([1,2,3]):
    df_voxel = pd.read_csv(things_path / filename_voxel_metadata.format(subject_id=subject_id))
    hemisphere_threshold = 36 #((df_voxel.voxel_x.max() + 1 ) / 2)
    df_roi_coordinates = build_voxel_roi_table(df_voxel, hemisphere_threshold=hemisphere_threshold)

    subject_folder = preprocessed_path / f"sub-0{subject_id}"
    subject_folder.mkdir(exist_ok=True)
    # Save table of roi coordinates
    df_roi_coordinates.to_parquet(subject_folder / "roi_coordinates.parquet", index=False)
    mask_size = get_voxel_size(df_voxel)

    # Load an example beta map
    beta_map = nib.load(things_path / filename_betas.format(subject_id=subject_id))
    assert beta_map.shape[:3] == mask_size # Check that the beta map has the same size as the mask

    mask = get_dense_mask(df_roi_coordinates, size=mask_size)

    # Save mask as npy
    np.save(subject_folder / "mask-glasser.npy", mask)

    # Save mask as nii
    mask_nii = nib.Nifti1Image(mask, beta_map.affine)
    nib.save(mask_nii, subject_folder / "mask-glasser.nii.gz")

    df_stimulus = pd.read_csv(things_path / filename_stimulus_metadata.format(subject_id=subject_id))
    with h5.File(things_path / filename_response_data.format(subject_id=subject_id), 'r') as file_betas:
        beta_tables = compute_voxel_tables(df_stimulus, df_roi_coordinates, file_betas)


    subject_folder_betas = subject_folder / "betas_roi"
    subject_folder_betas.mkdir(exist_ok=True)
    for roi, beta_table in beta_tables.items():
        np.save(subject_folder_betas / f"betas_glasser_roi_{roi}.npy", beta_table)

### Categories index

In [ ]:
category_ids = pd.read_csv(things_path / "stimuli/THINGS/Metadata/Concept-specific/unique_id.csv", header=None, names=["category_name"]).reset_index()
category_ids = category_ids.rename(columns={"index": "category_id"})

super_categories = pd.read_csv(things_path / "stimuli/THINGS/27 higher-level categories/category_mat_manual.tsv", sep="\t")
super_categories

category_ids = pd.concat([category_ids, super_categories], axis=1)

filename = "stimuli/THINGS/27 higher-level categories/categorization.tsv"

categorization = pd.read_csv(things_path / filename, sep="\t", header=None)

# Join all columns into a single string concatenated by a comma
categorization = categorization.apply(lambda x: ";".join(set(x.dropna())), axis=1)
category_ids['tags'] = categorization
category_ids
category_ids.to_csv(preprocessed_path / "categories.csv", index=False)

### Images index

In [72]:
images_folder = things_path / "stimuli/THINGS/Images"
images = list(images_folder.glob("**/*.jpg"))
# Sort images by name
images.sort(key=lambda x: x.name)

df_images = []
for things_id, image in enumerate(images):
    df_images.append(
        {"things_id": things_id, 
         "image_name": image.stem,
         "category": image.parent.name,
         "image_path": str(image.relative_to(things_path)),
        }
    )
df_images = pd.DataFrame(df_images)
categories = category_ids[['category_name', 'category_id']].set_index("category_name").to_dict("index")
df_images['category_id'] = df_images['category'].map(lambda x: categories[x]['category_id'])
df_images["image_category_index"] = df_images.groupby("category_id").cumcount() + 1
df_images = df_images[['things_id', 'image_name', 'category', 'category_id', 'image_category_index', 'image_path']]
# Save images table
df_images.to_csv(preprocessed_path / "images.csv", index=False)

### Captions

In [89]:
images_ids = df_images[['image_name', 'things_id', 'category_id', 'category']].set_index("image_name").to_dict("index")
df_captions = pd.read_csv(results_folder / "captions/captions-revised-things-pixtral-12b.csv")
df_captions['things_id_2'] = df_captions['name'].map(lambda x: images_ids[x]['things_id'])
df_captions['category_id'] = df_captions['name'].map(lambda x: images_ids[x]['category_id'])
df_captions['category'] = df_captions['name'].map(lambda x: images_ids[x]['category'])
df_captions = df_captions[['things_id_2', 'name', 'category', 'category_id', 'caption']]
df_captions = df_captions.rename(columns={"things_id_2": "things_id"})
df_captions.caption = df_captions.caption.astype("string")
df_captions.things_id = df_captions.things_id.astype("uint16")
df_captions.name = df_captions.name.astype("string")
df_captions.category = df_captions.category.astype("string").astype("category")
df_captions.category_id = df_captions.category_id.astype("uint8")
df_captions.to_csv(preprocessed_path / "captions.csv", index=False)

### Stimulus index

In [121]:

df_stimulus_index = []
for subject_id in [1,2,3]:
    df_stimulus = pd.read_csv(things_path / filename_stimulus_metadata.format(subject_id=subject_id))

    # Remove last .jpg from stimulus column
    df_stimulus.stimulus = df_stimulus.stimulus.str.replace(".jpg", "")
    # Rename stimulus column to image_name
    df_stimulus = df_stimulus.rename(columns={"stimulus": "image_name"})
    total_trials = len(df_stimulus)

    df_stimulus = df_stimulus.merge(df_images[['things_id', 'image_name', 'category_id', 'category', 'image_category_index']], on='image_name')
    assert total_trials == len(df_stimulus)

    df_stimulus['session_index'] = df_stimulus.groupby('session').cumcount()
    df_stimulus['run_index'] = df_stimulus.groupby('run').cumcount()
    df_stimulus  = df_stimulus.rename(columns={'trial_id': 'subject_index'})
    df_stimulus['repetition'] = df_stimulus.groupby('image_name').cumcount() + 1
    df_stimulus = df_stimulus[['subject_id', 'session', 'run', 'things_id', 'subject_index', 'session_index', 'run_index', 'repetition', 'image_name', 'category_id', 'category', 'image_category_index', 'trial_type']]
    df_stimulus_index.append(df_stimulus)
df_stimulus_index = pd.concat(df_stimulus_index)
df_stimulus_index.subject_id = df_stimulus_index.subject_id.astype("uint8")
df_stimulus_index.session = df_stimulus_index.session.astype("uint8")
df_stimulus_index.run = df_stimulus_index.run.astype("uint8")
df_stimulus_index.things_id = df_stimulus_index.things_id.astype("uint16")
df_stimulus_index.subject_index = df_stimulus_index.subject_index.astype("uint16")
df_stimulus_index.session_index = df_stimulus_index.session_index.astype("uint16")
df_stimulus_index.run_index = df_stimulus_index.run_index.astype("uint16")
df_stimulus_index.repetition = df_stimulus_index.repetition.astype("uint8")
df_stimulus_index.image_name = df_stimulus_index.image_name.astype("string")
df_stimulus_index.category_id = df_stimulus_index.category_id.astype("uint16")
df_stimulus_index.category = df_stimulus_index.category.astype("string").astype("category")
df_stimulus_index.image_category_index = df_stimulus_index.image_category_index.astype("uint8")
df_stimulus_index.trial_type = df_stimulus_index.trial_type.astype("string").astype("category")
df_stimulus_index = df_stimulus_index.reset_index(drop=True)

df_stimulus_index.to_parquet(preprocessed_path / "stimulus_index.parquet", index=False)
df_stimulus_index.to_csv(preprocessed_path / "stimulus_index.csv", index=False)
